In [0]:
import re
import numpy as np
import nltk
from tqdm import tqdm
from sklearn.utils import shuffle
import os
from gensim.models import word2vec
import pandas as pd
import yaml
from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
def clean_text(text):
      #Clean text by removing unnecessary characters and altering the format of words.

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:{}`+=~|.!?,]", "", text)
    
    return text

  # target?? context??



In [0]:
def dataset(filename):
  questions = list()
  stream = open( filename , 'rb')
  docs = yaml.safe_load(stream)
  conversations = docs['conversations']
  for con in conversations:
      if len( con ) > 2 :
          questions.append(con[0])
      elif len( con )> 1:
          questions.append(con[0])

  questions_c = []
  answers_c=[]
  for conve in questions:
      text = clean_text(conve)
      questions_c.append(text)

  
  return questions_c


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1tUTHFJsrBeCMVgRzmw0gI7Hn9aOVmc1M'})
download.GetContentFile('lonely.yml')
download = drive.CreateFile({'id': '1e10qngrE8AkB965LAu-iienU1Bapkncd'})
download.GetContentFile('generic1.yml')
download = drive.CreateFile({'id': '1_dTkZlzg4RWAGMl0Dh2QdY8nXC5dXLzi'})
download.GetContentFile('help.yml')
download = drive.CreateFile({'id': '1RT3slFT2iPhbsBL3H9J5v_f9eyR2BHEE'})
download.GetContentFile('greetings.yml')



In [5]:
x1 = dataset('lonely.yml')
y1 = ['lonely']*len(x1)

x2 = dataset('generic1.yml')
y2 = ['generic1']*len(x2)

x3 = dataset('help.yml')
y3 = ['help']*len(x3)

x4 = dataset('greetings.yml')
y4 = ['greetings']*len(x4)

v = TfidfVectorizer()
x = v.fit_transform(x1+x2+x3+x4)

print(x1+x2+x3+x4)

x = x.toarray()
print (x.shape)
y = pd.Series(y1+y2+y3+y4)

df = pd.DataFrame(x)
df = pd.concat([df,y],axis=1)
print (df.info())

['i feel sad', 'i feel sad', 'i am alone', 'thank you', 'you are very nice', 'you are very nice', 'everyone hates me', 'everybody hates me', 'will you be my friend', 'will you always be my friend', 'will you always take care of me', 'never leave me alone', 'never leave me', 'please do not leave me', 'do you feel alone', 'i do not have friends', 'i do not have friends', 'are you my friend', 'are you my friend', 'talk to me', 'i feel lonely', 'i am hurt', 'nobody likes me', 'nobody loves me', 'nobody takes care of me', 'nobody talks to me', 'what should i do', 'i want to die', 'i do not want to live', 'speak to me', 'what should we talk about', 'i am bored', 'i want to go out', 'i want a friend', 'are you lonely', 'i am tired', 'i am exhausted', 'what is your name', 'what is your age', 'what is your age', 'how old are you', 'how old are you', 'when were you born', 'when is your birthday', 'where do you live', 'what can you do', 'what do you eat', 'i love you', 'i like you', 'will you be 

In [9]:

from sklearn.naive_bayes import GaussianNB


clf = GaussianNB()
df = df.sample(frac=1).reset_index(drop=True)

clf.fit(df.iloc[:,:-1],df.iloc[:,-1])


GaussianNB(priors=None, var_smoothing=1e-09)

In [10]:
from sklearn.metrics import classification_report
print(classification_report(df.iloc[:,-1],clf.predict(df.iloc[:,:-1])))

              precision    recall  f1-score   support

    generic1       1.00      0.94      0.97       129
   greetings       1.00      1.00      1.00        25
        help       0.96      1.00      0.98        26
      lonely       0.82      0.97      0.89        37

    accuracy                           0.96       217
   macro avg       0.95      0.98      0.96       217
weighted avg       0.96      0.96      0.96       217



In [12]:
for _ in range(10):

  inp = input()
  inp2 = v.transform([inp])
  print(clf.predict(inp2.toarray()))

hello
['greetings']
i need help
['help']
what is this
['greetings']
hello
['greetings']
what is your age
['generic1']
what is your name
['generic1']


KeyboardInterrupt: ignored

In [13]:
from sklearn.externals import joblib 
joblib.dump(clf, 'category_clf.pkl') 
joblib.dump(v,'vectorizer.pkl')

from google.colab import files

files.download('category_clf.pkl') 
files.download('vectorizer.pkl') 



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
